In [1]:
import os

Create DataIngestionConfig file to specify a class that hold configdata of data ingestion

In [2]:
from dataclasses import dataclass
from pathlib import Path

#dataclass is a class that introduced to specially deal with data. frozen=True means this class is frozen to only take defined parameters
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
print(CONFIG_FILE_PATH)
print(PARAMS_FILE_PATH)

../config/config.yaml
../params.yaml


Create ConfigurationManager

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    # returning configurations related to data ingestion
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


Let's create DataIngestion Class

In [6]:
import os
import urllib.request as request
import zipfile
import gdown
from cnnClassifier.utils import logger
from cnnClassifier.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # download file from the location
    def download_file(self) -> str:
        """Fetch data from the URL
        """

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} to the file {zip_download_dir}")

            file_id = dataset_url.split("/")[5]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    # extract the zipfile
    def extract_zip_file(self):
        """
        zip file path: str
        Extracts the zip file into the data directory
        Function returns none
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

Creating data ingestion pipeline

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-28 08:58:47,502: INFO: common: yaml file: ../config/config.yaml loaded successfully]
[2024-03-28 08:58:47,848: INFO: common: yaml file: ../params.yaml loaded successfully]
[2024-03-28 08:58:47,851: INFO: common: created directory at artifacts]
[2024-03-28 08:58:47,852: INFO: common: created directory at artifacts/data_ingestion]
[2024-03-28 08:58:47,853: INFO: 3837693123: Downloading data from https://drive.google.com/file/d/1SXbuSXHYYL1pwItse2uNUWfCcW-2_De-/view?usp=sharing to the file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1SXbuSXHYYL1pwItse2uNUWfCcW-2_De-
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1SXbuSXHYYL1pwItse2uNUWfCcW-2_De-&confirm=t&uuid=bc63aece-bdac-46c1-80f1-286fc467010e
To: /home/kaiz/Desktop/End-to-End-Chest-Cancer-Classification-Using-MLFlow---DVC/research/artifacts/data_ingestion/data.zip
100%|██████████| 49.0M/49.0M [00:20<00:00, 2.39MB/s]

[2024-03-28 08:59:11,546: INFO: 3837693123: Downloaded data from https://drive.google.com/file/d/1SXbuSXHYYL1pwItse2uNUWfCcW-2_De-/view?usp=sharing into file artifacts/data_ingestion/data.zip]
